# Kepler Framework, Emulate v15

In [1]:
from kepler.pandas.menu       import ElectronSequence as Chain
from kepler.pandas.readers    import load, load_in_loop, load_hdf
from core import Efficiency
from Gaugi import mkdir_p, progressbar
import numpy as np
import pandas as pd
import collections
import os
from pprint import pprint
from copy import deepcopy

import gc
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

Welcome to JupyROOT 6.16/00
Using all sub packages with ROOT dependence
INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.

Applying ATLAS style settings...

Applying ATLAS style settings...

Applying ATLAS style settings...


In [2]:
output_path = 'output/emulation'
mkdir_p(output_path)

In [3]:
dpath_data17 = '../../data/output/data17_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.dataframe.h5'
dpath_data18 = '../../data/output/data18_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.dataframe.h5'
dpath_mc16   = '../../data/output/mc16_13TeV.302236_309995_341330.sgn.boosted_probes.WZ_llqq_plus_radion_ZZ_llqq_plus_ggH3000.merge.dataframe.h5'

In [4]:
tpath_data17 = 'output/emulation/data17_table_v15.h5'
tpath_data18 = 'output/emulation/data18_table_v15.h5'
tpath_mc16   = 'output/emulation/mc16_table_v15.h5'

## Setup Chains:

In [5]:
# create my chain
chains = [
            Chain( "HLT_e17_lhvloose_nod0_ringer_v15_L1EM15VHI", L1Seed = 'L1_EM15VHI' , l2calo_column = 'ringer_v15_vloose'),
            Chain( "HLT_e28_lhtight_nod0_ringer_v15_ivarloose", L1Seed = 'L1_EM22VHI' , l2calo_column = 'ringer_v15_tight' ),
            Chain( "HLT_e60_lhmedium_nod0_ringer_v15_L1EM24VHI", L1Seed = 'L1_EM24VHI' , l2calo_column = 'ringer_v15_medium'),
            Chain( "HLT_e140_lhloose_nod0_ringer_v15" , L1Seed = 'L1_EM24VHI', l2calo_column = 'ringer_v15_loose'),
]

def emulate( df ):
    for chain in progressbar(chains, prefix='Emulate...'):
        chain.apply(df)

OrderedDict([('L1Seed', 'L1_EM15VHI'),
             ('L2Calo', (14000.0, 'ringer_v15_vloose')),
             ('L2', 'trig_L2_el_cut_pt15to20'),
             ('EFCalo', 17000.0),
             ('HLT', (17000.0, 'trig_EF_el_lhvloose'))])
OrderedDict([('L1Seed', 'L1_EM22VHI'),
             ('L2Calo', (25000.0, 'ringer_v15_tight')),
             ('L2', 'trig_L2_el_cut_pt20to50'),
             ('EFCalo', 28000.0),
             ('HLT', (28000.0, 'trig_EF_el_lhtight_ivarloose'))])
OrderedDict([('L1Seed', 'L1_EM24VHI'),
             ('L2Calo', (57000.0, 'ringer_v15_medium')),
             ('L2', 'trig_L2_el_cut_pt50toInf'),
             ('EFCalo', 60000.0),
             ('HLT', (60000.0, 'trig_EF_el_lhmedium'))])
OrderedDict([('L1Seed', 'L1_EM24VHI'),
             ('L2Calo', (137000.0, 'ringer_v15_loose')),
             ('L2', 'trig_L2_el_cut_pt50toInf'),
             ('EFCalo', 140000.0),
             ('HLT', (140000.0, 'trig_EF_el_lhloose'))])


## Emulate 2017:

Take something like 1 hour

In [6]:
data17_df = pd.concat( (load_hdf(dpath_data17), load_hdf(tpath_data17)), axis=1)

In [7]:
emulate(data17_df)

Emulate...: 100%|██████████| 4/4 [01:34<00:00, 23.63s/it]


In [8]:
data17_df.shape

(43311283, 111)

### Validate:

In [9]:
eff_data17_Zee = Efficiency( output_path+'/data17_Zee_efficiency_v15.root')

2022-03-13 09:38:14,133 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data17_Zee_efficiency_v15.root


In [10]:
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e17_lhvloose_nod0_ringer_v15_L1EM15VHI', pidname='el_lhvloose', reset=True)
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e28_lhtight_nod0_ringer_v15_ivarloose' , pidname='el_lhtight' , reset=True)
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e60_lhmedium_nod0_ringer_v15_L1EM24VHI', pidname='el_lhmedium', reset=True)
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e140_lhloose_nod0_ringer_v15'          , pidname='el_lhloose' , reset=True)

Filling...: 100%|██████████| 5/5 [01:52<00:00, 22.45s/it]


HLT_e17_lhvloose_nod0_ringer_v15_L1EM15VHI
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.1069  | 30158981/32047563 |
| L2Calo | 93.3838  | 29927232/32047563 |
|   L2   | 93.1454  | 29850823/32047563 |
| EFCalo | 91.8365  | 29431365/32047563 |
|  HLT   | 89.9499  | 28826743/32047563 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [01:21<00:00, 16.35s/it]


HLT_e28_lhtight_nod0_ringer_v15_ivarloose
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.6937  | 21557594/22765602 |
| L2Calo | 94.2390  | 21454080/22765602 |
|   L2   | 94.1236  | 21427815/22765602 |
| EFCalo | 93.1988  | 21217260/22765602 |
|  HLT   | 87.0303  | 19812979/22765602 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [00:05<00:00,  1.18s/it]


HLT_e60_lhmedium_nod0_ringer_v15_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.5079  | 1361055/1367786 |
| L2Calo | 97.1808  | 1329226/1367786 |
|   L2   | 97.0905  | 1327990/1367786 |
| EFCalo | 94.9967  | 1299352/1367786 |
|  HLT   | 91.7378  | 1254777/1367786 |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 20.15it/s]

HLT_e140_lhloose_nod0_ringer_v15
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 100.0000 | 60541/60541  |
| L2Calo | 98.0228  | 59344/60541  |
|   L2   | 97.9518  | 59301/60541  |
| EFCalo | 96.6287  | 58500/60541  |
|  HLT   | 94.6549  | 57305/60541  |
+--------+----------+--------------+


In [11]:
eff_data17_Zee.save()

In [12]:
eff_data17_JF17 = Efficiency( output_path+'/data17_JF17_efficiency_v15.root')

2022-03-13 09:42:26,104 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data17_JF17_efficiency_v15.root


In [13]:
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e17_lhvloose_nod0_ringer_v15_L1EM15VHI', reset=True)
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e28_lhtight_nod0_ringer_v15_ivarloose' , reset=True)
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e60_lhmedium_nod0_ringer_v15_L1EM24VHI', reset=True)
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e140_lhloose_nod0_ringer_v15'          , reset=True)

Filling...: 100%|██████████| 5/5 [00:22<00:00,  4.42s/it]


HLT_e17_lhvloose_nod0_ringer_v15_L1EM15VHI
+--------+----------+------------------+
|  Step  | Eff [%%] |   passed/total   |
+--------+----------+------------------+
| L1Calo | 84.4275  | 8738241/10349992 |
| L2Calo |  3.8566  | 399154/10349992  |
|   L2   |  3.4932  | 361543/10349992  |
| EFCalo |  3.2718  | 338629/10349992  |
|  HLT   |  0.1166  |  12069/10349992  |
+--------+----------+------------------+


Filling...: 100%|██████████| 5/5 [00:19<00:00,  3.94s/it]


HLT_e28_lhtight_nod0_ringer_v15_ivarloose
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 90.6060  | 8384948/9254300 |
| L2Calo |  0.6747  |  62442/9254300  |
|   L2   |  0.6280  |  58115/9254300  |
| EFCalo |  0.4910  |  45437/9254300  |
|  HLT   |  0.0015  |   136/9254300   |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:16<00:00,  3.33s/it]


HLT_e60_lhmedium_nod0_ringer_v15_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.9012  | 7844422/7852181 |
| L2Calo |  0.7712  |  60556/7852181  |
|   L2   |  0.7146  |  56111/7852181  |
| EFCalo |  0.5394  |  42357/7852181  |
|  HLT   |  0.0019  |   153/7852181   |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:12<00:00,  2.48s/it]

HLT_e140_lhloose_nod0_ringer_v15
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.9227  | 5629684/5634041 |
| L2Calo |  2.1598  |  121682/5634041 |
|   L2   |  1.9948  |  112386/5634041 |
| EFCalo |  1.7153  |  96643/5634041  |
|  HLT   |  0.0222  |   1248/5634041  |
+--------+----------+-----------------+


In [14]:
eff_data17_JF17.save()

In [15]:
del data17_df

## Emulate 2018:

In [16]:
data18_df = pd.concat( (load_hdf(dpath_data18), load_hdf(tpath_data18)), axis=1)

In [17]:
emulate(data18_df)

Emulate...: 100%|██████████| 4/4 [01:33<00:00, 23.48s/it]


In [18]:
data18_df.shape

(41873973, 111)

In [19]:
eff_data18_Zee = Efficiency( output_path+'/data18_Zee_efficiency_v15.root')

2022-03-13 09:47:40,118 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data18_Zee_efficiency_v15.root


In [20]:
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e17_lhvloose_nod0_ringer_v15_L1EM15VHI', pidname='el_lhvloose', reset=True)
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e28_lhtight_nod0_ringer_v15_ivarloose' , pidname='el_lhtight' , reset=True)
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e60_lhmedium_nod0_ringer_v15_L1EM24VHI', pidname='el_lhmedium', reset=True)
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e140_lhloose_nod0_ringer_v15'          , pidname='el_lhloose' , reset=True)

Filling...: 100%|██████████| 5/5 [01:52<00:00, 22.46s/it]


HLT_e17_lhvloose_nod0_ringer_v15_L1EM15VHI
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.1301  | 30705693/32620495 |
| L2Calo | 93.4266  | 30476227/32620495 |
|   L2   | 93.1936  | 30400211/32620495 |
| EFCalo | 91.8621  | 29965860/32620495 |
|  HLT   | 90.4455  | 29503769/32620495 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [01:22<00:00, 16.42s/it]


HLT_e28_lhtight_nod0_ringer_v15_ivarloose
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.6555  | 22067825/23313826 |
| L2Calo | 94.2023  | 21962151/23313826 |
|   L2   | 94.1064  | 21939796/23313826 |
| EFCalo | 93.3069  | 21753405/23313826 |
|  HLT   | 87.5940  | 20421506/23313826 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [00:05<00:00,  1.19s/it]


HLT_e60_lhmedium_nod0_ringer_v15_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.5563  | 1375545/1381675 |
| L2Calo | 97.1972  | 1342950/1381675 |
|   L2   | 97.1080  | 1341717/1381675 |
| EFCalo | 95.0187  | 1312850/1381675 |
|  HLT   | 92.0488  | 1271815/1381675 |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 20.17it/s]


HLT_e140_lhloose_nod0_ringer_v15
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 99.9934  | 60385/60389  |
| L2Calo | 98.0427  | 59207/60389  |
|   L2   | 97.9433  | 59147/60389  |
| EFCalo | 96.7577  | 58431/60389  |
|  HLT   | 94.9444  | 57336/60389  |
+--------+----------+--------------+


In [21]:
eff_data18_Zee.save()

In [22]:
eff_data18_JF17 = Efficiency( output_path+'/data18_JF17_efficiency_v15.root')

2022-03-13 09:51:53,599 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data18_JF17_efficiency_v15.root


In [23]:
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e17_lhvloose_nod0_ringer_v15_L1EM15VHI', reset=True)
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e28_lhtight_nod0_ringer_v15_ivarloose' , reset=True)
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e60_lhmedium_nod0_ringer_v15_L1EM24VHI', reset=True)
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e140_lhloose_nod0_ringer_v15'          , reset=True)

Filling...: 100%|██████████| 5/5 [00:17<00:00,  3.59s/it]


HLT_e17_lhvloose_nod0_ringer_v15_L1EM15VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 85.5183  | 7240922/8467098 |
| L2Calo |  4.2584  |  360562/8467098 |
|   L2   |  3.8400  |  325136/8467098 |
| EFCalo |  3.6218  |  306663/8467098 |
|  HLT   |  0.5592  |  47344/8467098  |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:15<00:00,  3.19s/it]


HLT_e28_lhtight_nod0_ringer_v15_ivarloose
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 91.4008  | 6972786/7628804 |
| L2Calo |  1.0130  |  77280/7628804  |
|   L2   |  0.9688  |  73908/7628804  |
| EFCalo |  0.8367  |  63831/7628804  |
|  HLT   |  0.0429  |   3270/7628804  |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:13<00:00,  2.74s/it]


HLT_e60_lhmedium_nod0_ringer_v15_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.8995  | 6555389/6561985 |
| L2Calo |  1.0962  |  71934/6561985  |
|   L2   |  1.0424  |  68401/6561985  |
| EFCalo |  0.8752  |  57430/6561985  |
|  HLT   |  0.0525  |   3446/6561985  |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:10<00:00,  2.07s/it]

HLT_e140_lhloose_nod0_ringer_v15
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.9159  | 4772502/4776518 |
| L2Calo |  2.5696  |  122736/4776518 |
|   L2   |  2.3906  |  114188/4776518 |
| EFCalo |  2.1230  |  101404/4776518 |
|  HLT   |  0.2888  |  13796/4776518  |
+--------+----------+-----------------+


In [24]:
eff_data18_JF17.save()

## Evaluate mc16 boosted:

In [25]:
mc16_df = pd.concat( (load_hdf(dpath_mc16), load_hdf(tpath_mc16)), axis=1)

In [26]:
emulate(mc16_df)

Emulate...: 100%|██████████| 4/4 [00:00<00:00, 30.09it/s]


In [27]:
eff_mc16 = Efficiency( output_path+'/mc16_boosted_efficiency_v15.root')

2022-03-13 09:53:10,148 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/mc16_boosted_efficiency_v15.root


In [28]:
eff_mc16.fill( mc16_df.loc[mc16_df['target']==1], 'HLT_e17_lhvloose_nod0_ringer_v15_L1EM15VHI', pidname='el_lhvloose', reset=True)
eff_mc16.fill( mc16_df.loc[mc16_df['target']==1], 'HLT_e28_lhtight_nod0_ringer_v15_ivarloose' , pidname='el_lhtight' , reset=True)
eff_mc16.fill( mc16_df.loc[mc16_df['target']==1], 'HLT_e60_lhmedium_nod0_ringer_v15_L1EM24VHI', pidname='el_lhmedium', reset=True)
eff_mc16.fill( mc16_df.loc[mc16_df['target']==1], 'HLT_e140_lhloose_nod0_ringer_v15'          , pidname='el_lhloose' , reset=True)

Filling...: 100%|██████████| 5/5 [00:00<00:00, 15.88it/s]


HLT_e17_lhvloose_nod0_ringer_v15_L1EM15VHI
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 99.7303  | 78407/78619  |
| L2Calo | 98.7522  | 77638/78619  |
|   L2   | 98.7420  | 77630/78619  |
| EFCalo | 77.4177  | 60865/78619  |
|  HLT   | 76.3543  | 60029/78619  |
+--------+----------+--------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 17.46it/s]


HLT_e28_lhtight_nod0_ringer_v15_ivarloose
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 99.8427  | 72990/73105  |
| L2Calo | 91.9075  | 67189/73105  |
|   L2   | 91.9007  | 67184/73105  |
| EFCalo | 72.9704  | 53345/73105  |
|  HLT   | 70.3153  | 51404/73105  |
+--------+----------+--------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 16.98it/s]


HLT_e60_lhmedium_nod0_ringer_v15_L1EM24VHI
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 99.9635  | 73911/73938  |
| L2Calo | 95.2460  | 70423/73938  |
|   L2   | 95.2393  | 70418/73938  |
| EFCalo | 74.7924  | 55300/73938  |
|  HLT   | 73.0288  | 53996/73938  |
+--------+----------+--------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 17.78it/s]

HLT_e140_lhloose_nod0_ringer_v15
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 99.9882  | 67947/67955  |
| L2Calo | 98.7330  | 67094/67955  |
|   L2   | 98.7286  | 67091/67955  |
| EFCalo | 76.7375  | 52147/67955  |
|  HLT   | 75.3351  | 51194/67955  |
+--------+----------+--------------+


In [29]:
eff_mc16.save()

## 